In [ ]:
import sys
sys.path.append('../')
import numpy as np
import healpy as hp
import torch
from wlgan.model import Generator, compute_avg_mat
import pickle
import os
from tqdm.auto import tqdm, trange

In [ ]:
nside = 512
gen_nside = 4 * nside
lmax = 2 * gen_nside

num_bins = 4

mask = np.ones(hp.nside2npix(nside), dtype=bool)

nmocks = 28

In [ ]:
lognorm_params = pickle.load(open('lognorm_params.pkl', 'rb'))

cl = lognorm_params['cl']
y_cl = lognorm_params['y_cl']
shift = lognorm_params['shift']
mu = lognorm_params['mu']

In [ ]:
device = torch.device('cpu')

avg_mat = compute_avg_mat(nside, mask).to(device)

ckpt = torch.load('ckpt.pt', map_location=device)
gen = Generator(num_bins, avg_mat).to(device)
gen.load_state_dict(ckpt['gen_ema'])

In [ ]:
def generate_lognorm_map(nside):
    alm = hp.synalm(y_cl, lmax=lmax, new=False)
    k = np.exp(mu[:,None] + hp.alm2map(alm, gen_nside, pol=False)) - shift[:,None]
    k = hp.ud_grade(k, nside)
    return k

os.system('rm -rf gan_mocks')
os.system('mkdir -p gan_mocks')

os.system('rm -rf lognorm_mocks')
os.system('mkdir -p lognorm_mocks')

for i in trange(nmocks):
    with torch.no_grad():
        k_ln = generate_lognorm_map(nside)
        x_ln = torch.tensor(hp.reorder(k_ln, r2n=True))[None,...].to(device, dtype=torch.float32) / 0.01
        x_gan = gen(x_ln)
        k_gan = hp.reorder((x_gan * 0.01).view(num_bins,-1).cpu().numpy(), n2r=True)
        np.save(f'gan_mocks/mock_{i}.npy', k_gan)
        np.save(f'lognorm_mocks/mock_{i}.npy', k_ln)